In [3]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!ls "/content/drive/My Drive"

In [ ]:
Загрузка данных с гугл диска

### Считывание данных из Google drive


In [4]:
df = pd.read_csv('/content/drive/My Drive/hw_abt_2_byuser_old.csv', sep = '\t', index_col = 0)
df

,group_id,metric_value,users,visits,churn_users,churn_visits,group
0,8,6630.000,0,0,0,8,A
1,13,6933.976,0,0,1,6,B
2,10,8261.976,1,10,1,8,B
3,4,5522.000,1,9,1,7,A
4,16,5075.976,1,6,0,8,B
...,...,...,...,...,...,...,...
4995,6,3666.000,1,4,1,5,A
4996,12,6464.976,0,0,1,8,B
4997,1,4335.000,0,0,0,8,A
4998,13,6307.976,0,0,1,8,B


### Разбивка всего data_frame на 2 по подгруппам A и B

In [5]:
df_groupA = df[df['group'] == 'A']
df_groupB = df[df['group'] == 'B']

### Использование разных метрик

In [6]:
# Метрика metric_value - близка к нормальному распределению, но не является таковой
ser1 = df_groupA['metric_value']
ser2 = df_groupB['metric_value']

# метрика2 - не является нормальной
ser1 = df_groupA['churn_users']
ser2 = df_groupB['churn_users']

# метрика3
# ser1 = df_groupA['churn_visits']
# ser2 = df_groupB['churn_visits']

# метрика4 - нормальна, значит для нее можно использовать соответствующие критерии
# ser1 = df_groupA['churn_visits'] / df_groupA['visits'] 
# ser2 = df_groupB['churn_visits'] / df_groupB['visits'] 

# ser1 = (df_groupA['visits'] - df_groupA['churn_visits']) / df_groupA['visits'] 
# ser2 = (df_groupA['visits'] - df_groupB['churn_visits']) / df_groupB['visits'] 


### Графики разных метрик

In [ ]:
import matplotlib.pyplot as plt
plt.hist(ser1)
plt.hist(ser2)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
pd.Series(ser1).plot.box()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
pd.Series(ser2).plot.box()

## Использование разных критериев

### Критерий Шапиро - проверка на нормальность распределений по группам A и B

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import shapiro

alpha = 0.05

st1 = shapiro(pd.Series(ser1))
print('Distributions is {}normal\n'.format( {True:'not ', False:''}[st1[1] < alpha]))

st2 = shapiro(pd.Series(ser2))
print('Distributions is {}normal\n'.format( {True:'not ', False:''}[st2[1] < alpha]))


Distributions is not normal

Distributions is not normal



### Критерий Левена для нормальных метрик для равенства дисперсий (не подойдет)

In [ ]:
import numpy as np
from scipy.stats import levene

alpha = 0.05 

def levene_test(df1, df2, p_value = alpha): 
	st = levene(df1, df2) 
	print('Variances of distributions is {}equal\n'.format( {True:'not ', False:''}[st[1] < p_value])) 
levene_test(ser1, ser2)


Variances of distributions is equal



### U-критерий Манна-Уитни для обычных метрик(не подойдет). Мощность выше, чем у Q-критерия Розенбаума

In [ ]:
from numpy.random import randn
from scipy.stats import mannwhitneyu

stat, p = mannwhitneyu(ser1, ser2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
alpha = 0.05

if p > alpha: print('Same distribution (fail to reject H0).Тк одинаковые медианы, то нововведение не вызвало эффектов.')
else: print('Different distribution (reject H0). Тк разные медианы, то нововведение вызвало эффекты.')



### H-критерий Краскела-Уоллиса для обычных метрик для проверки равества медиан


In [ ]:
import pandas
from scipy.stats import mstats

H, pval = mstats.kruskalwallis(ser1.array, ser2.array)

print("H-statistic:", H)
print("P-Value:", pval)
if pval < 0.05: print("Reject H0 - Significant differences exist between groups.")
else: print("Accept H0 - No significant difference between groups.")


H-statistic: 437.5572316307385
P-Value: 3.682053429972744e-97
Reject H0 - Significant differences exist between groups.


### F-критерий Фишера для нормальных метрик для проверки равенства дисперсий (не подойдет)

In [ ]:
import numpy as np
from scipy.stats import f_oneway

alpha = 0.05 
p_value = f_oneway(ser1, ser2)[1] 
print('{} the H0: Var(X1) == Var(X2)\n'.format( {False:'Reject', True:'Confirm'}[p_value > alpha]))

Reject the H0: Var(X1) == Var(X2)



### Критерий Ансари для обычных метрик для провеки равенства дисперсий. (Подтверждаем равенство дисперсий в ответе)

In [7]:
from scipy.stats import ansari

alpha = 0.05 
p_value = ansari(ser1, ser2)[1] 
print('{} the H0: Var(X1) == Var(X2)\n'.format( {False:'Reject', True:'Confirm'}[p_value < alpha]))


Reject the H0: Var(X1) == Var(X2)



### T-критерий Стьюдента для проверки на статзначимость




In [8]:
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind
import matplotlib.pyplot as plt
%matplotlib inline
alpha = 0.05

st = ttest_ind(pd.Series(ser1), pd.Series(ser2))
 
print('Diffs is {}significant\n'.format( {False:'not ',True:''}[st[1] < alpha])) 
print(st[1], alpha, "\n")
print('Тк статзначима, то изменения существенны\n')

Diffs is significant

1.4725094742968288e-101 0.05 

Тк статзначима, то изменения существенны



### Расчёт необходимого объёма выборки (используем t-test)


In [ ]:
import pandas.util.testing as tm
import numpy as np
from statsmodels.stats.power import TTestIndPower

# parameters for power analysis

alpha = 0.05 # Альфа стандартно 5%
power = 0.8  # можем менять значения на 0.9, 0.95 и тд

effect = 0.08 # Эффект в 8%
ser1 = df['metric_value'][(df['group']=='A')]
ser2 = df['metric_value'][(df['group']=='B')]
analysis = TTestIndPower()
result = analysis.solve_power(effect, power=power, nobs1=None, ratio=1.0, alpha=alpha)
print('Sample Size: %.3f' % result)
print(f'Текущий размер выборок в A группе {len(ser1)}')
print(f'Текущий размер выборок в B группе {len(ser2)}\n')

effect = 0.1 # Эффект в 10%
ser1 = df['churn_users'][(df['group']=='A')]
ser2 = df['churn_users'][(df['group']=='B')]
analysis = TTestIndPower()
result = analysis.solve_power(effect, power=power, nobs1=None, ratio=1.0, alpha=alpha)
print('Sample Size: %.3f' % result)
print(f'Текущий размер выборок в A группе {len(ser1)}')
print(f'Текущий размер выборок в B группе {len(ser2)}\n')

Sample Size: 2453.730
Текущий размер выборок в A группе 2451
Текущий размер выборок в B группе 2549

Sample Size: 1570.733
Текущий размер выборок в A группе 2451
Текущий размер выборок в B группе 2549



#### Так как у меня получилось, что текущий размер выборок почти такой же как sample size, то эксперимент можно прекращать. Для второй метрики получилось, что текущий размер выборок даже больше, чем sample size. Тоже можно прекращать.

#### Не использовала критерий Бартлетта, тк критерий Левена менее чувствителен к отклонениям от нормальности. Критерий Левена тоже не подходит. Не использовала T-критерий Уилкоксона, тк нужно равенство размеров выборок. Не использовала критерий Колмогорова-Смирнова, тк критерий Шапиро-Уилка имеет бОльшую мощность.